# Test model

### original file

https://github.com/openai/gpt-2-output-dataset/blob/master/baseline.py

## Before running this notebook

1. Create /output folder
   1. Insert all crawled dataset(csv)
   1. Rename them as same with GPT dataset files.
1. Create /log folder

## Difference between baseline

- Using Word2Vec instead of TF-IDF Vectorizer
- Using Gridsearch to find best parameter

In [1]:
!pip install gensim
!pip install scikit-learn

In [60]:
# import packages

import os
import csv
import json
import time

import numpy as np

from scipy import sparse

from sklearn.model_selection import PredefinedSplit, GridSearchCV, train_test_split, KFold
from sklearn.linear_model import LogisticRegression

from gensim.models import Word2Vec

In [61]:
# create tokenizer
# example code from https://github.com/SKT-AI/KoGPT2

from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token="</s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [62]:
SHORT_MODE = False

In [63]:
# load data + preprocessing

def load_data(data_dir, crawled_dir, source, max_len=250):
    path = os.path.join(data_dir, "{}.csv".format(source))
    crawled_path = os.path.join(crawled_dir, "{}.csv".format(source))
    dataset = list(csv.reader(open(path, encoding="utf8")))
    crawled_dataset = list(csv.reader(open(crawled_path, encoding="cp949")))
    
    try:
        path2 = os.path.join(data_dir, "{}2.csv".format(source))
        dataset2 = list(csv.reader(open(path2, encoding="utf8")))
        dataset.extend(dataset2)
    except:
        pass
    
    n = len(dataset)
    
    length = min(max_len, min(50, n) if SHORT_MODE else n)

    texts = []
    labels = [1, 0] * length

    for data in dataset[:length]:
        idx = int(round(float(data[0])))
        tokens = tokenizer.tokenize(data[5])
        texts.append(' '.join(tokens))
        tokens = tokenizer.tokenize(crawled_dataset[idx][4])
        texts.append(' '.join(tokens))
    return texts, labels

In [64]:
# main function

def main(
    data_dir="data/",
    crawl_dir="output/",
    log_dir="log/",
    topics=["culture", "economy", "it_science", "politics", "society", "world"],
    train_test_ratio=0.1,
):
    texts_list, labels_list = [], []
    for topic in topics:
        texts, labels = load_data(data_dir, crawl_dir, topic, 750)
        texts_list.extend(texts)
        labels_list.extend(labels)

    texts_train, texts_test, labels_train, labels_test = train_test_split(
        texts_list, labels_list, test_size=train_test_ratio, random_state=42, shuffle=True,
    )
    n_train, n_test = len(texts_train), len(texts_test)
    
    w2v_model = Word2Vec(sentences=texts_list, vector_size=512, window=9, min_count=5, workers=4, sg=0)
    train_features = [w2v_model.wv.get_mean_vector(y) for y in texts_train]
    test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_test]

    model = LogisticRegression(max_iter=20)
    params = {'C': [1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8, 16, 32, 64]}
    split = PredefinedSplit([-1]*n_train+[0]*n_test)
    search = GridSearchCV(model, params, cv=split, refit=False)
    search.fit(sparse.vstack([train_features, test_features]), labels_train + labels_test)
    model = model.set_params(**search.best_params_)
    model.fit(train_features, labels_train)
    test_accuracy = model.score(test_features, labels_test) * 100.0

    result = model.predict(test_features)
    result_proba = model.predict_proba(test_features)
    result_log_proba = model.predict_log_proba(test_features)
    kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
    for res, pred in zip(result, labels_test):
        if res == 1:
            kind["tp" if res == pred else "fp"] += 1
        else:
            kind["tn" if res == pred else "fn"] += 1
    precision = kind["tp"] / (kind["tp"] + kind["fp"])
    recall = kind["tp"] / (kind["tp"] + kind["fn"])

    ce_loss = 0
    for label, value in zip(labels_test, result_log_proba):
        ce_loss -= label * value[1] + (1 - label) * value[0]
    ce_loss /= len(labels_test)
    data = {
        "test_accuracy": test_accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "F_score": 2 * precision * recall / (precision + recall),
        "mse_loss": np.sum(np.array(labels_test) - model.predict_proba(test_features)[:, 1]) ** 2 / len(labels_test),
        "ce_loss": ce_loss,
        "param": search.best_params_,
        "label_and_result": list(zip(labels_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
    }
    print(data)
    json.dump(data, open(os.path.join(log_dir, f"result_{str(round(time.time() * 1000))}.json"), "w"), indent=4)

In [65]:
# main function

def main_overfit(
    data_dir="data/",
    crawl_dir="output/",
    log_dir="log/",
    topics=["culture", "economy", "it_science", "politics", "society", "world"],
    train_test_ratio=0.1,
):
    texts_list, labels_list = [], []
    for topic in topics:
        texts, labels = load_data(data_dir, crawl_dir, topic, 750)
        texts_list.extend(texts)
        labels_list.extend(labels)

    texts_train, texts_test, labels_train, labels_test = train_test_split(
        texts_list, labels_list, test_size=train_test_ratio, random_state=42, shuffle=True,
    )
    
    w2v_model = Word2Vec(sentences=texts_list, vector_size=512, window=9, min_count=5, workers=4, sg=0)
    train_features = [w2v_model.wv.get_mean_vector(y) for y in texts_train]
    test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_test]

    model = LogisticRegression(penalty=None, max_iter=100000)
    model.fit(train_features, labels_train)
    test_accuracy = model.score(test_features, labels_test) * 100.0

    result = model.predict(test_features)
    result_proba = model.predict_proba(test_features)
    result_log_proba = model.predict_log_proba(test_features)
    kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
    for res, pred in zip(result, labels_test):
        if res == 1:
            kind["tp" if res == pred else "fp"] += 1
        else:
            kind["tn" if res == pred else "fn"] += 1
    precision = kind["tp"] / (kind["tp"] + kind["fp"])
    recall = kind["tp"] / (kind["tp"] + kind["fn"])

    ce_loss = 0
    for label, value in zip(labels_test, result_log_proba):
        ce_loss -= label * value[1] + (1 - label) * value[0]
    ce_loss /= len(labels_test)
    data = {
        "test_accuracy": test_accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "F_score": 2 * precision * recall / (precision + recall),
        "mse_loss": np.sum(np.array(labels_test) - model.predict_proba(test_features)[:, 1]) ** 2 / len(labels_test),
        "ce_loss": ce_loss,
        "label_and_result": list(zip(labels_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
    }
    print(data)
    json.dump(data, open(os.path.join(log_dir, f"result_{str(round(time.time() * 1000))}_overfit.json"), "w"), indent=4)

In [66]:
# main function

def main_kfold(
    data_dir="data/",
    crawl_dir="output/",
    log_dir="log/",
    topics=["culture", "economy", "it_science", "politics", "society", "world"],
    train_test_ratio=0.1,
):
    texts_list, labels_list = [], []
    for topic in topics:
        texts, labels = load_data(data_dir, crawl_dir, topic, 750)
        texts_list.extend(texts)
        labels_list.extend(labels)

    texts_list_train, texts_list_test, labels_list_train, labels_list_test = train_test_split(
        texts_list, labels_list, test_size=train_test_ratio, random_state=42, shuffle=True,
    )
    
    w2v_model = Word2Vec(sentences=texts_list_train, vector_size=512, window=9, min_count=5, workers=4, sg=0)
    
    kf = KFold(n_splits=5)
    models = []
    param_list = []
    
    for idx, (train, test) in enumerate(kf.split(texts_list_train)):
        texts_train, texts_test = [texts_list_train[x] for x in train], [texts_list_train[x] for x in test]
        labels_train, labels_test = [labels_list_train[x] for x in train], [labels_list_train[x] for x in test]

        n_train, n_test = len(train), len(test)
        
        train_features = [w2v_model.wv.get_mean_vector(y) for y in texts_train]
        test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_test]

        model = LogisticRegression(max_iter=100)
        params = {'C': [1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8, 16, 32, 64]}
        split = PredefinedSplit([-1]*n_train+[0]*n_test)
        search = GridSearchCV(model, params, cv=split, refit=False)
        search.fit(sparse.vstack([train_features, test_features]), labels_train + labels_test)
        model = model.set_params(**search.best_params_)
        model.fit(train_features, labels_train)
        test_accuracy = model.score(test_features, labels_test) * 100.0
        
        models.append(model)
        param_list.append(search.best_params_)

        result = model.predict(test_features)
        result_proba = model.predict_proba(test_features)
        result_log_proba = model.predict_log_proba(test_features)
        kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
        for res, pred in zip(result, labels_test):
            if res == 1:
                kind["tp" if res == pred else "fp"] += 1
            else:
                kind["tn" if res == pred else "fn"] += 1
        precision = kind["tp"] / (kind["tp"] + kind["fp"])
        recall = kind["tp"] / (kind["tp"] + kind["fn"])

        ce_loss = 0
        for label, value in zip(labels_test, result_log_proba):
            ce_loss -= label * value[1] + (1 - label) * value[0]
        ce_loss /= len(labels_test)
        data = {
            "test_accuracy": test_accuracy,
            "test_precision": precision,
            "test_recall": recall,
            "F_score": 2 * precision * recall / (precision + recall),
            "mse_loss": np.sum(np.array(labels_test) - model.predict_proba(test_features)[:, 1]) ** 2 / len(labels_test),
            "ce_loss": ce_loss,
            "param": search.best_params_,
            "label_and_result": list(zip(labels_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
        }
        print(data)
        json.dump(data, open(os.path.join(log_dir, f"result_{str(round(time.time() * 1000))}_{idx}.json"), "w"), indent=4)
    
    # soft voting
    results = []
    results_proba = []
    results_log_proba = []
    test_features = [w2v_model.wv.get_mean_vector(y) for y in texts_list_test]
    for model in models:
        result = model.predict(test_features)
        result_proba = model.predict_proba(test_features)
        result_log_proba = model.predict_log_proba(test_features)
        
        results.append(result)
        results_proba.append(result_proba)
        results_log_proba.append(result_log_proba)
    
    results = np.array(results)
    results_proba = np.array(results_proba)
    results_log_proba = np.array(results_log_proba)
    
    proba = np.mean(results_proba, axis=0)
    result = np.argmax(proba, axis=1)
    log_proba = np.log(proba)
    
    kind = {"tp": 0, "fp": 0, "fn": 0, "tn": 0}
    for res, pred in zip(result, labels_list_test):
        if res == 1:
            kind["tp" if res == pred else "fp"] += 1
        else:
            kind["tn" if res == pred else "fn"] += 1
    precision = kind["tp"] / (kind["tp"] + kind["fp"])
    recall = kind["tp"] / (kind["tp"] + kind["fn"])

    ce_loss = 0
    for label, value in zip(labels_list_test, log_proba):
        ce_loss -= label * value[1] + (1 - label) * value[0]
    ce_loss /= len(labels_list_test)
    data = {
        "test_accuracy": test_accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "F_score": 2 * precision * recall / (precision + recall),
        "mse_loss": np.sum(np.array(labels_list_test) - proba[:, 1]) ** 2 / len(labels_list_test),
        "ce_loss": ce_loss,
        "param": param_list,
        "label_and_result": list(zip(labels_list_test, model.predict(test_features).tolist(), result_proba[:, 1].tolist())),
    }
    print("Final:", data)
    json.dump(data, open(os.path.join(log_dir, f"result_{str(round(time.time() * 1000))}_total.json"), "w"), indent=4)

In [67]:
# run main function

# main(train_test_ratio=0.2)
# main_overfit(train_test_ratio=0.2)
main_kfold()

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

{'test_accuracy': 90.61728395061729, 'test_precision': 0.9166666666666666, 'test_recall': 0.8888888888888888, 'F_score': 0.9025641025641026, 'mse_loss': 0.31162431660107165, 'ce_loss': 0.2696374193058447, 'param': {'C': 64}, 'label_and_result': [(1, 1, 0.9574031781878897), (1, 1, 0.9771649626509096), (0, 0, 0.14183220662412546), (0, 0, 0.2448220053035184), (0, 0, 0.36478673306905507), (1, 1, 0.9690036056868515), (0, 0, 0.027073181393710292), (0, 0, 0.015884458096491784), (1, 1, 0.9400330197082865), (1, 1, 0.6855717935586214), (0, 0, 0.052235350740795286), (1, 1, 0.9384416886460584), (1, 1, 0.9285117325814347), (1, 0, 0.27653802955331325), (1, 1, 0.9378636293525636), (0, 0, 0.1318880149384342), (0, 0, 0.033066615899473684), (1, 1, 0.7250580956385874), (1, 1, 0.94215009545339), (1, 1, 0.7181648525216916), (0, 0, 0.1710575869165971), (1, 1, 0.967517172206773), (0, 0, 0.0995160926986208), (0, 0, 0.09605696331674604), (1, 0, 0.03283491686205686), (1, 0, 0.28134801983597413), (0, 0, 0.171460

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

{'test_accuracy': 90.24691358024691, 'test_precision': 0.9232804232804233, 'test_recall': 0.87468671679198, 'F_score': 0.8983268983268983, 'mse_loss': 0.0007666660436503973, 'ce_loss': 0.28997719538230915, 'param': {'C': 64}, 'label_and_result': [(0, 0, 0.13895267835643757), (1, 0, 0.08549052760584623), (0, 0, 0.04469477424104374), (0, 0, 0.09591483827659618), (0, 0, 0.06816055952842662), (1, 1, 0.9413936527802643), (0, 0, 0.19804140947942525), (0, 0, 0.02370771634962652), (0, 0, 0.38728131203599414), (0, 0, 0.471834460860163), (0, 0, 0.21799957847599086), (0, 0, 0.2564087389381361), (0, 1, 0.6207607332756373), (0, 0, 0.4375991301934008), (0, 0, 0.02681568046034662), (0, 0, 0.4287077231118685), (0, 0, 0.43745213480333667), (0, 0, 0.06890550549687512), (1, 1, 0.8672096872622442), (1, 1, 0.8610667541580221), (1, 1, 0.814502270290796), (1, 1, 0.992921687594649), (1, 1, 0.9562655591334066), (1, 1, 0.6251463762468635), (1, 1, 0.9203306847302488), (1, 1, 0.98429805815723), (1, 1, 0.986120218

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

{'test_accuracy': 88.08641975308642, 'test_precision': 0.9141742522756827, 'test_recall': 0.846987951807229, 'F_score': 0.8792995622263916, 'mse_loss': 0.2527052152897154, 'ce_loss': 0.30711656957327677, 'param': {'C': 64}, 'label_and_result': [(0, 0, 0.04792649681503285), (1, 1, 0.9622395777628763), (1, 1, 0.8650973566574512), (0, 0, 0.23162107333073037), (1, 0, 0.17588840219468121), (1, 1, 0.9918031683275829), (0, 0, 0.3187860286668379), (0, 0, 0.3342788666879841), (0, 1, 0.5724436679913405), (1, 1, 0.9027992580721752), (1, 1, 0.8860144364458131), (1, 0, 0.03652546617093071), (0, 0, 0.03877808195374763), (0, 0, 0.05608414628612149), (0, 0, 0.020246356375817516), (0, 0, 0.28470575394412745), (0, 0, 0.42842899234912885), (0, 0, 0.008827838601965388), (0, 0, 0.050040640690253525), (1, 0, 0.17004228289722215), (1, 1, 0.9827409984539979), (1, 0, 0.21782471447295026), (1, 0, 0.08005397895723079), (0, 0, 0.19818653508853742), (1, 1, 0.9886227085122694), (1, 1, 0.8509605685056688), (0, 0, 0.

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

{'test_accuracy': 90.8641975308642, 'test_precision': 0.9187418086500655, 'test_recall': 0.8907242693773825, 'F_score': 0.904516129032258, 'mse_loss': 0.18667795860566713, 'ce_loss': 0.2750968963806273, 'param': {'C': 64}, 'label_and_result': [(1, 1, 0.9146374085383407), (1, 1, 0.9683893513568606), (0, 0, 0.0381564902592505), (1, 1, 0.8165283843438769), (1, 1, 0.8839183995385003), (0, 0, 0.21579205273096302), (1, 1, 0.979294519930785), (1, 1, 0.824276765351164), (1, 1, 0.9019165115680857), (0, 0, 0.4211342790481799), (1, 1, 0.9490747179489328), (1, 1, 0.9601252146299722), (1, 1, 0.8192646029558384), (0, 0, 0.11978385233252521), (0, 0, 0.1437691483571405), (1, 1, 0.8139399300648454), (1, 1, 0.9146437060997407), (0, 0, 0.019177677766817898), (1, 1, 0.9910121360902181), (0, 0, 0.0373840895131445), (1, 1, 0.9056192161441026), (1, 1, 0.9627434457913893), (0, 0, 0.14379210818183302), (0, 0, 0.42992558086326216), (0, 0, 0.21127757263482821), (0, 0, 0.2607729132209493), (1, 1, 0.71277718343110

e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\스꾸\4-1\인공지능프로젝트-박호건\aip\w2v_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

{'test_accuracy': 90.12345679012346, 'test_precision': 0.933588761174968, 'test_recall': 0.8712753277711561, 'F_score': 0.9013563501849569, 'mse_loss': 0.16338906468633635, 'ce_loss': 0.2953647402281979, 'param': {'C': 64}, 'label_and_result': [(0, 0, 0.3503965163202933), (0, 0, 0.18765750911121543), (0, 0, 0.12095149490753189), (1, 1, 0.9873178911151084), (1, 1, 0.8877083052559354), (1, 1, 0.7675952965761141), (0, 0, 0.22855410501738155), (1, 1, 0.7978676615807405), (1, 0, 0.3832652165439924), (0, 0, 0.05576225825005843), (1, 1, 0.9968699649972242), (1, 1, 0.7524445004216688), (0, 0, 0.45875330352013255), (1, 1, 0.8689462568137223), (1, 1, 0.9582905393509129), (0, 0, 0.08743098561476619), (0, 0, 0.48811178808030875), (0, 0, 0.06598497499651008), (1, 1, 0.6188754282337994), (0, 0, 0.2323035665549405), (0, 0, 0.03892356353294963), (1, 1, 0.806584168940662), (0, 1, 0.7749643701839042), (1, 0, 0.48105020539823906), (1, 1, 0.5057875892709308), (0, 0, 0.005481060757650513), (0, 1, 0.5329902